In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split


from tensorflow.keras.applications import VGG19,VGG16,Xception
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image

from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout


In [ ]:
no_dir='/kaggle/input/brian-tumor-dataset/Brain Tumor Data Set/Brain Tumor Data Set/Healthy'
yes_dir='/kaggle/input/brian-tumor-dataset/Brain Tumor Data Set/Brain Tumor Data Set/Brain Tumor'

In [ ]:
def create_class_dataframe(class_dir, class_name):
    files = os.listdir(class_dir)
    image_files = [file for file in files if file.lower().endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp'))]
    data = {'image_path': [os.path.join(class_dir, file) for file in image_files],
            'label': [class_name] * len(image_files)}
    return pd.DataFrame(data)

In [ ]:
no_df=create_class_dataframe(no_dir, '0')
yes_df=create_class_dataframe(yes_dir, '1')

In [ ]:
no_df.shape,yes_df.shape

In [ ]:
df = pd.concat([no_df,yes_df], ignore_index=True)

In [ ]:
def shuffle_dataframe(df):
    shuffled_df = df.sample(frac=1).reset_index(drop=True)
    return shuffled_df

df=shuffle_dataframe(df)

In [ ]:
df.head()

In [ ]:
df.label.value_counts()

In [ ]:
df.label.value_counts().plot(kind='bar')

In [ ]:
def print_class_proportions(labels):
    # Count the occurrences of each class label
    class_counts = labels.value_counts()
    
    # Calculate proportions
    total_samples = len(labels)
    class_proportions = class_counts / total_samples
    
    # Print class proportions
    print("Class Proportions:")
    for class_label, proportion in class_proportions.items():
        print(f"Class {class_label}: {proportion:.2%}")

# Example usage:
# Assuming 'labels' is a Pandas Series containing class 

print_class_proportions(df['label'])

In [ ]:
#img=cv2.imread(df['Image_Path'].values[0])
img = mpimg.imread(df['image_path'].values[0])

plt.imshow(img)
plt.axis('off')  
plt.show()

In [ ]:

def is_grayscale(image_path):
    # Read the image
    image = cv2.imread(image_path)

    # Check if image is None
    if image is None:
        print("Error: Unable to read the image.")
        return False

    # Get the number of channels in the image
    num_channels = image.shape[2] if len(image.shape) == 3 else 1

    # Return True if the image has only 1 channel (grayscale), False otherwise
    return num_channels == 1

# Example usage
image_path = df['image_path'].values[2]
if is_grayscale(image_path):
    print("The image is grayscale.")
else:
    print("The image is RGB.")


In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score
import seaborn as sns
import numpy as np

def evaluate_classification(model, test_generator, classes):
    # Generate predictions for the test data
    y_pred = model.predict(test_generator)
    
    # Convert predicted probabilities to class labels
    predictions = np.argmax(y_pred, axis=1)
    
    # Get true labels from the test generator
    y_true = test_generator.classes
    
    # Calculate accuracy
    accuracy = accuracy_score(y_true, predictions)
    
    # Calculate precision, recall, and F1-score
    precision = precision_score(y_true, predictions, average=None)
    recall = recall_score(y_true, predictions, average=None)
    f1 = f1_score(y_true, predictions, average=None)
    
    # Calculate confusion matrix
    cm = confusion_matrix(y_true, predictions)
    
    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()
    
    # Print evaluation metrics
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1-score:", f1)

# Assuming you have defined your model and test_generator
classes = ['tumor', 'no_tumor']  # Specify your class labels


# test

In [ ]:
df_0=df.query('label=="0"').sample(100)
df_1=df.query('label=="1"').sample(100)

df_test = pd.concat([df_0,df_1], ignore_index=True)


In [ ]:
df_test=shuffle_dataframe(df_test)
df_test.head()

In [ ]:
df_test.shape

In [ ]:

# Merge the DataFrames with indicator
merged_df = df.merge(df_test, how='outer', indicator=True)

# Keep only the records that exist in df1 but not in df2
result_df = merged_df[merged_df['_merge'] == 'left_only']

# Drop the '_merge' column if needed
result_df = result_df.drop(columns=['_merge'])

print(result_df.shape)


In [ ]:
result_df.shape,df.shape,df_test.shape

In [ ]:
result_df=shuffle_dataframe(result_df)
result_df.head()

# Augmentation

In [ ]:
train_df, valid_df = train_test_split(result_df, stratify=result_df['label'],test_size=0.2, random_state=42)


In [ ]:

datagen = ImageDataGenerator(
    rotation_range=30,  
    shear_range=0.2,  
    zoom_range=0.1,  
    horizontal_flip=True,
        vertical_flip=True,
    #brightness_range=[0.4,0.6],
     
        rescale=1.0/255,

    
    fill_mode='nearest'  # Strategy used for filling in newly created pixels
)

train_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='image_path',
    y_col='label',
   target_size=(224, 224),
    batch_size=64,
     color_mode="rgb",
    class_mode='binary'
)

valid_generator = datagen.flow_from_dataframe(
    dataframe=valid_df,
    x_col='image_path',
    y_col='label',
    target_size=(224, 224),
    batch_size=64,
     color_mode="rgb",
    class_mode='binary'
)


test_generator = datagen.flow_from_dataframe(
    df_test,  
    x_col='image_path',
    y_col='label',
    target_size=(224, 224),  #
    batch_size=64,
    color_mode="rgb",
    class_mode='binary',  # or 'categorical' depending on your task
    ) 

# model

In [ ]:
batch_size=64
epochs=20

In [ ]:
import tensorflow as tf
import keras_cv
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Softmax
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Check and set GPU memory growth
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)

In [ ]:
def create_cnn_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(128, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        #Dropout(.2),

        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        # Dropout(.2),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),

        Flatten(),
        Dense(64, activation='relu'),
        
        
        #Dropout(.2),
         #Dense(64, activation='relu'), 
         #Dropout(.2),
        Dense(16, activation='relu'),
      #Dropout(0.2),
        Dense(num_classes, activation='sigmoid')
    ])
    
    return model


In [ ]:
model=create_cnn_model((224,224,3),1)
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
#early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
history = model.fit(
    train_generator,
    batch_size=batch_size,    epochs=epochs,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples //batch_size)
         #,callbacks=[early_stopping])

# Optionally, you can plot training history
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracies')
plt.legend()
plt.show()

In [ ]:
loss,acc=model.evaluate(test_generator)
loss,acc

In [ ]:
evaluate_classification(model, test_generator, classes)


In [ ]:
from sklearn.metrics import precision_score, f1_score, confusion_matrix

# Step 3: Predict probabilities for the test data
y_pred_probs = model.predict(test_generator)
# Convert probabilities to class labels
y_pred = np.argmax(y_pred_probs, axis=1)

# Get true labels
y_true = test_generator.classes

# Compute precision
precision = precision_score(y_true, y_pred, average='weighted')

# Compute F1-score
f1 = f1_score(y_true, y_pred, average='weighted')

# Compute confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)

print("Precision:", precision)
print("F1-score:", f1)
print("Confusion Matrix:")
print(conf_matrix)


In [ ]:
base_model = Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# Freeze convolutional layers
for layer in base_model.layers:
    layer.trainable = False
# Add custom classification layers
x = Flatten()(base_model.output)
x = Dense(64, activation='relu')(x)
#x = Dropout(.2)(x)

x = Dense(32, activation='relu')(x)
#x = Dropout(.1)(x)

#x = Dense(16, activation='relu')(x)
#x = Dropout(.2)(x)

#x = Dense(16, activation='relu')(x)
#x = Dense(8, activation='relu')(x)

predictions = Dense(1, activation='sigmoid')(x)
# Create final model
model_Xception = Model(inputs=base_model.input, outputs=predictions)
# Compile the model
model_Xception.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

#early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model_Xception.fit(train_generator, 
                       steps_per_epoch=train_generator.samples // batch_size,
                       epochs=epochs,
                       batch_size=batch_size,
                       validation_data=valid_generator,
                      validation_steps=valid_generator.samples //batch_size)
 #                      ,callbacks=[early_stopping])
# Plot training and validation accuracies
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Model Training and Validation Accuracy')
plt.legend()
plt.show()

In [ ]:
loss,acc=model_Xception.evaluate(test_generator)
loss,acc

In [ ]:
model_Xception.save('brain_xception.h5')

In [ ]:
model_Xception.summary()

In [ ]:
evaluate_classification(model_Xception, test_generator, classes)
